In [1]:
import numpy as np
import pandas as pd 
import sys
import os
from nash_equilibrium.nash_solver import milp_max_sym_ent_2p, replicator_dynamics_nash, milp_nash_2p
import pygambit

Matplotlib created a temporary cache directory at /var/folders/fh/fwc37qhn04d8sxp65hwv1kxm0000gn/T/matplotlib-7z3v6fvl because the default path (/Users/gabesmithline/.matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [2]:
matrix = "/Users/gabesmithline/Desktop/caif_negotiation/test_matrices_performance_matrices/performance_matrix_0.csv"
game_matrix_2 = "'meta_game_analysis/game_matrix_2_100_bootstrap/csv/performance_matrix.csv"
performance_matrix = pd.read_csv(matrix, index_col=0)
payoff_matrix = performance_matrix.values
agents = performance_matrix.index.tolist()

In [3]:
performance_matrix

,anthropic_3.7_sonnet_2025-02-19_circle_5,anthropic_3.7_sonnet_2025-02-19_circle_6,anthropic_sonnet_3.7_reasoning_2025-02-19_circle_0,gemini_2.0_flash_circle_2,gemini_2.0_flash_circle_5,openai_4o_2024-08-06_circle_4,openai_4o_2024-08-06_circle_5,openai_4o_2024-08-06_circle_6,openai_o3_mini_2025-01-31_circle_0
anthropic_3.7_sonnet_2025-02-19_circle_5,505.316907,389.841379,441.109434,586.527838,463.117742,535.820513,609.416279,539.964634,550.555556
anthropic_3.7_sonnet_2025-02-19_circle_6,537.147241,554.136276,473.953171,495.094737,463.107941,663.525385,630.832000,602.463462,584.484750
anthropic_sonnet_3.7_reasoning_2025-02-19_circle_0,542.563396,432.204146,496.895313,604.389111,467.763902,672.619706,454.183636,546.409111,593.853864
gemini_2.0_flash_circle_2,501.015946,592.210000,521.362222,494.653398,491.497872,451.561290,563.152143,582.493333,465.030222
gemini_2.0_flash_circle_5,539.252903,468.902647,564.572927,519.750638,532.880260,494.707073,523.029762,639.344500,588.016977
openai_4o_2024-08-06_circle_4,480.110769,526.606667,427.662059,662.154839,552.726829,548.685053,658.030851,701.954054,510.547368
openai_4o_2024-08-06_circle_5,592.361628,567.304222,577.080909,660.710714,465.343810,578.785745,611.763469,565.616364,497.661667
openai_4o_2024-08-06_circle_6,646.404634,455.032692,532.803778,559.917667,446.255250,653.362973,582.700303,575.515625,597.430556
openai_o3_mini_2025-01-31_circle_0,586.251111,520.210750,522.585455,567.686667,437.153023,548.840000,793.041389,628.189444,565.100500


In [4]:
def compute_regret(strategy, payoff_matrix):
    """
    Compute the regret for each agent given a strategy.
    
    Args:
        strategy: Nash equilibrium strategy vector
        payoff_matrix: Game payoff matrix
        
    Returns:
        regret: Vector of regrets for each agent
        nash_value: Expected utility when Nash plays against itself
    """
    # Expected utilities when playing against the Nash strategy
    expected_utils = payoff_matrix @ strategy
    
    # Nash equilibrium value (expected utility when Nash plays against itself)
    nash_value = strategy @ payoff_matrix @ strategy
    
    # Calculate regret for each agent
    regret = expected_utils - nash_value
    
    return regret, nash_value, expected_utils

def is_epsilon_nash(strategy, payoff_matrix, epsilon=0.05):
    """
    Check if a strategy is an epsilon-Nash equilibrium.
    
    Args:
        strategy: Strategy vector to check
        payoff_matrix: Game payoff matrix
        epsilon: Epsilon value for Nash equilibrium
        
    Returns:
        bool: True if it's an epsilon-Nash equilibrium
    """
    regret, nash_value, _ = compute_regret(strategy, payoff_matrix)
    max_regret = np.max(regret)
    
    print(f"Maximum regret: {max_regret:.6f}")
    print(f"Nash value: {nash_value:.6f}")
    
    return max_regret <= epsilon, max_regret, nash_value

In [5]:
print("Computing Nash equilibria...")
rd_nash, _ = replicator_dynamics_nash(payoff_matrix, max_iter=10000, epsilon=.05)



me_nash = milp_max_sym_ent_2p(payoff_matrix)
print("\nReplicator Dynamics Nash Equilibrium:")
print(pd.DataFrame({
    'Agent': agents,
    'Probability': rd_nash
}))

print("\nMaximum Entropy Nash Equilibrium:")
print(pd.DataFrame({
    'Agent': agents,
    'Probability': me_nash
}))

# Check if the equilibria are close to each other


Computing Nash equilibria...
Running nashpy replicator dynamics implementation...
Running custom replicator dynamics implementation...
  Custom reached max iterations with regret 32.330056 at start point 0
  Custom reached max iterations with regret 141.087727 at start point 1
  Custom reached max iterations with regret 38.073724 at start point 2
  Custom reached max iterations with regret 80.185597 at start point 3
  Custom reached max iterations with regret 167.501441 at start point 4
  Custom reached max iterations with regret 19.846569 at start point 5
  Custom reached max iterations with regret 123.934653 at start point 6
  Custom reached max iterations with regret 181.277920 at start point 7
  Custom reached max iterations with regret 126.438429 at start point 8
  Custom reached max iterations with regret 32.330056 at start point 9
  Custom reached max iterations with regret 32.330056 at start point 10
  Custom reached max iterations with regret 32.330056 at start point 11
  Cust

In [8]:
milp_strat = milp_nash_2p(performance_matrix, .05)
milp_regret, _, _ = compute_regret(milp_strat, performance_matrix)
max(milp_regret)



1.3839439816365484e-08

In [9]:
# Check if they are 0.05-Nash equilibria
print("\nChecking if RD Nash is a 0.05-Nash equilibrium:")
rd_is_nash, rd_max_regret, rd_value = is_epsilon_nash(rd_nash, payoff_matrix, 0.05)
print(f"RD Nash is a 0.05-Nash equilibrium: {rd_is_nash}")

print("\nChecking if ME Nash is a 0.05-Nash equilibrium:")
me_is_nash, me_max_regret, me_value = is_epsilon_nash(me_nash, payoff_matrix, 0.05)
print(f"ME Nash is a 0.05-Nash equilibrium: {me_is_nash}")

print("\nChecking if Plain MILP Nash is a 0.05-Nash equilibrium:")
milp_is_nash, milp_max_regret, milp_value = is_epsilon_nash(milp_strat, payoff_matrix, 0.05)
print(f"MILP Nash is a 0.05-Nash equilibrium: {me_is_nash}")



# Compute and display individual regrets for both strategies
rd_regret, _, rd_expected_utils = compute_regret(rd_nash, payoff_matrix)
me_regret, _, me_expected_utils = compute_regret(me_nash, payoff_matrix)
milp_regret, _, milp_expected_utils = compute_regret(milp_strat, payoff_matrix)

print("\nRegrets for each agent under RD Nash strategy:")
print(pd.DataFrame({
    'Agent': agents,
    'Expected Utility': rd_expected_utils,
    'Regret': rd_regret
}))

print("\nRegrets for each agent under ME Nash strategy:")
print(pd.DataFrame({
    'Agent': agents,
    'Expected Utility': me_expected_utils,
    'Regret': me_regret
}))


print("\nRegrets for each agent under MILP Nash strategy:")
print(pd.DataFrame({
    'Agent': agents,
    'Expected Utility': milp_expected_utils,
    'Regret': milp_regret
}))



# Compare the three equilibria
''''
print("\nComparison of RD Nash and ME Nash:")
print(f"RD Nash Value: {rd_value:.6f}")
print(f"ME Nash Value: {me_value:.6f}")
print(f"RD Nash Max Regret: {rd_max_regret:.6f}")
print(f"ME Nash Max Regret: {me_max_regret:.6f}")
print(f"Difference in Nash Value: {abs(rd_value - me_value):.6f}")
print(f"Difference in Max Regret: {abs(rd_max_regret - me_max_regret):.6f}")

# Calculate the L1 distance between the two strategies
l1_distance = np.sum(np.abs(rd_nash - me_nash))
print(f"L1 distance between RD Nash and ME Nash: {l1_distance:.6f}")
'''


Checking if RD Nash is a 0.05-Nash equilibrium:
Maximum regret: 0.862802
Nash value: 568.387465
RD Nash is a 0.05-Nash equilibrium: False

Checking if ME Nash is a 0.05-Nash equilibrium:
Maximum regret: 0.000000
Nash value: 543.406878
ME Nash is a 0.05-Nash equilibrium: True

Regrets for each agent under RD Nash strategy:
                                               Agent  Expected Utility  \
0           anthropic_3.7_sonnet_2025-02-19_circle_5        454.053912   
1           anthropic_3.7_sonnet_2025-02-19_circle_6        569.250267   
2  anthropic_sonnet_3.7_reasoning_2025-02-19_circ...        466.806421   
3                          gemini_2.0_flash_circle_2        564.121976   
4                          gemini_2.0_flash_circle_5        498.252083   
5                      openai_4o_2024-08-06_circle_4        546.436988   
6                      openai_4o_2024-08-06_circle_5        564.233002   
7                      openai_4o_2024-08-06_circle_6        500.441557   
8        